# AUC-ROC

In [ ]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt

from sklearn.utils import resample
from sklearn.datasets import load_breast_cancer, load_iris, make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import confusion_matrix, plot_confusion_matrix,\
    precision_score, recall_score, accuracy_score, f1_score, log_loss,\
    roc_curve, roc_auc_score, classification_report, plot_roc_curve

# Objectives

- Calculate and interpret probability estimates
- Adjust the threshold of a logistic regression model
- Visualize, calculate and interpret the AUC-ROC metric

# Motivation

Now that we've learned how to evaluate a classification model's predictions, let's dig deeper to see how else we might evaluate our models and how we can use that information to improve them.

# Scenario: Identifying Heart Disease

Let's use [this Kaggle dataset](https://www.kaggle.com/ronitf/heart-disease-uci) about predicting heart disease.

In [ ]:
hd_data = pd.read_csv('heart.csv')
hd_data.info()

In [ ]:
# Separate data into feature and target DataFrames
hd_X = hd_data.drop('target', axis = 1)
hd_y = hd_data['target']

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(hd_X, hd_y, test_size=.25,
                                                   random_state=1)
# Scale the data for modeling
hd_scaler = StandardScaler()
hd_scaler.fit(X_train)
X_train_sc = hd_scaler.transform(X_train)
X_test_sc = hd_scaler.transform(X_test)

# Train a logistic regresssion model with the train data
hd_model = LogisticRegression(random_state=42)
hd_model.fit(X_train_sc, y_train)

## Predicting Labels

Let's look at some predictions from our example.

In [ ]:
y_pred = hd_model.predict(X_test_sc)
y_pred[:10]

When we run the `.predict()` method, `sklearn` gives us the predicted values for each transaction in our test set: 0 if predicting "not fraudulent", 1 if predicting "fraudulent"

## Probability Estimates

If you remember how the logistic regression model works, though, it doesn't actually generate predicted values of 0 or 1. It creates an S-shaped curve to approximate the data, estimating the _probability_ that they belong to the target class. This probability takes a value _between_ 0 and 1.

![](https://www.graphpad.com/guides/prism/latest/curve-fitting/images/hmfile_hash_38a8acae.png)

Source: [GraphPad](https://www.graphpad.com/guides/prism/latest/curve-fitting/reg_simple_logistic_and_linear_difference.htm)

We can get these estimated probabilities using the `.predict_proba()` method. Each element gives two probabilities: the estimate probability of being in the 0 class (not heart disease) and the 1 class (heart disease)

In [ ]:
y_prob = hd_model.predict_proba(X_test_sc)
y_prob[:5]

## Thresholds

How did we get those 0 and 1 label predictions, when the model only calculates probabilities between 0 and 1? 

Now the default behavior is simply to take the larger of these values as the "real" prediction. Since $0.996 > 0.004$, we'll understand the model to be predicting this point to belong to class "0" (or the negative class). An equivalent way of understanding the default behavior is that we:

- round the predicted numbers up to 1 if they are at least as large as 0.5; and
- round them down to 0 if they are less than 0.5.

Since the probabilities must sum to 1, there will never be any problem with this algorithm. We refer to this value of 0.5 as the **threshold**.

But we don't have to do things this way. Suppose we're building a model that predicts the presence of cancer from X-ray scans. And suppose we get a pair of probabilities for some particular scan that look like this:

- pred_neg: 0.52, pred_pos: 0.48

Because false negatives (cancers not flagged) are *much* more costly than false positives (non-cancers flagged as cancers), we may well want to **adjust our threshold**. We might want to have our model predict "positive" if the corresponding probability is, say, as low as 0.4, or maybe even as low as 0.1. (Speaking for myself, if there was even a 10% chance that I had cancer, I think I'd probably want to know about it.) 

## True & False Positive Rates

Adjusting the threshold can increase or decrease performance on different evaluation metrics. When doing this, data scientist often look at changes in two metrics: **True Positive Rate (TPR)** and **False Positive Rate (FPR)**. Let's define and calculate these. 

To do this, we'll first need to get the values from the confusion matrix.

In [ ]:
cm = confusion_matrix(y_test, hd_model.predict(X_test_sc))

In [ ]:
cm

In [ ]:
tp, tn, fp, fn = cm[1][1], cm[0][0], cm[0][1], cm[1][0]

### True Positive Rate

True Positive Rate (TPR) is the same as recall, measuring how many of the positive cases we correctly classified as positive.

**True Positive Rate (TPR)** = **Recall** = $\frac{TP}{TP + FN}$

How many of the patients with heart disease did my model identify?

In [ ]:
tpr = tp / (tp + fn)
print(tpr)

### False Positive Rate

False Positive Rate (FPR) measures how many of the negative casses we incorrectly classified as positive.

**False Positive Rate (TPR)** = $\frac{FP}{FP + TN}$

How many of the patients without heart disease did my model flag as having heart disease?

In [ ]:
fpr = fp / (fp + tn)
print(fpr)

## Adjusting the threshold

The true- and false-positive rates will change if we make adjustments to the threshold. In fact, in the present case that was the whole point of making the adjustment: We want to minimize our false negatives.

So this is how the plot of these rates takes shape.

Let's build a function that will take in our data, together with a threshold setting, and return the corresponding true- and false-positive rates.

# The Receiver Operating Characteristic (ROC) Curve

The Receiver Operating Characteristic (ROC) curve plots the true-positive rate vs. the false-positive rate. Let's define these now:

In [ ]:
def classify_rates(y_test, y_probs, model, thresh):
    y_hat = []
    for val in y_probs:                                       # Each element in y_hat_probs is an array.
        if val[0] < thresh:                                 # We'll set our own threshold for classifying
            y_hat.append(1)                                 # a test point as positive! The lower my threshold,
        else:                                               # the fewer predicted positives I'll have. For the
            y_hat.append(0)                                 # cancer example, I'd want to set a *high* threshold.
    cm = confusion_matrix(y_test, y_hat)
    tp, tn, fp, fn = cm[1][1], cm[0][0], cm[0][1], cm[1][0]
    tpr = tp / (tp + fn)
    fpr = fp / (fp + tn)
    return tpr, fpr, f'tpr:{round(tpr, 3)}, fpr:{round(fpr, 3)}'

True- and false-positive rates for various thresholds:

In [ ]:
for x in np.linspace(0, 1, 11):
    print(f'Rates at threshold = {round(x, 2)}: ' + classify_rates(y_test, y_prob, hd_model, x)[2])

As my threshold goes up, I'll have more positive predictions, which means I'll have both more true positives and more false positives.

Note:

- I can artificially increase my true-positive rate to 1 by setting my threshold to 1, but at that point my false-positive rate is also 1! I'll have no true negatives and no false negatives. This will arise naturally if my training data has **very few (actual) negatives**. 
- I can artificially reduce my false-positive rate to 0 by setting my threshold to 0, but at  that point my true-positive rate is also 0! I'll have no true positives and no false positives. This will arise naturally if my training data has **very few (actual)  positives**. 

## Area Under the Curve (AUC)

The ROC curve will be a plot of tpr (on the y-axis) vs. fpr (on the x-axis). There will always be a point at (0, 0) and another at (1, 1). The question is what happens in the middle. Since we want our y-values to be as high as possible for any particular x-value, a natural metric is to calculate the **area under the curve**. The larger the area, the better the classifier. The maximum possible area is the area of the whole box between 0 and 1 on both axes, so that's a **maximum area of 1**.

What's the minimum? Well that depends on the ratios of (actual) positive and negatives in my data, in much the way that a baseline accuracy score does.

Remember: If my test data comprises 90% positives and only 10% negatives, then a simple classifier that always predicts "positive" will be 90% accurate! And so that would be the baseline level for a classifier on that data.

If we have equal numbers of positives and negatives, then we can set an **absolute minimum area of 0.5**. That's the "curve" we'd get by plotting a straight diagonal line from (0, 0) to (1, 1).

Why? The area under the curve really represents the test's ability to **discriminate** positives from negatives. Suppose I randomly took several pairs of points, one positive and one negative, and checked my test's predictions. The area under the curve represents a threshold-independent measure of how often my test would get the two predictions correct.

## Plotting the Curve

Let's plot our own ROC curve. We'll create an array of different thresholds and use our `classify_rates()` function to get the true- and false-positive rates for each threshold.

One way of choosing a threshold **independently of business concerns** is to select the point on the curve that is furthest from (1, 0), the "worse-case" point where our true-positive rate is 0 and our false-positive rate is 1. So let's find that point as well:

In [ ]:
tprs = []
fprs = []
diffs = []
for x in np.linspace(0, 1, 101):
    fprs.append(classify_rates(y_test, y_prob, hd_model, x)[1])
    tprs.append(classify_rates(y_test, y_prob, hd_model, x)[0])
    diffs.append(np.sqrt(tprs[-1]**2 + (1-fprs[-1])**2))
    
max_dist = diffs.index(np.max(diffs))
print(f"""With a threshold of {(max_dist - 1) / 100}: \n"""
      f"""\tYou\'ll have a True Positive Rate of {round(tprs[max_dist], 3)} \n"""
      f"""\tand a False Positive Rate of {round(fprs[max_dist], 3)}""")

fig, ax = plt.subplots(figsize=(8, 6))
ax.plot(fprs[:max_dist], tprs[:max_dist], 'r.')
ax.plot(fprs[max_dist], tprs[max_dist], 'ko', ms=10)
ax.plot(fprs[max_dist + 1:], tprs[max_dist + 1:], 'r.')
ax.plot(fprs, fprs, '.');

Scikit-Learn's `roc_auc_score()` function will compute the area under the curve for us:

In [ ]:
# Extract the probabilitiy predictions for the "1" class (heart disease)
y_hat_hd = y_prob[:, 1]

roc_auc_score(y_test, y_hat_hd)

Let's compare our curve with scikit-learn's:

In [ ]:
# Get the FPR and TPR data
fpr, tpr, thresholds = roc_curve(y_test, y_hat_hd)

# Plot the FPR and TPR data
fig, ax = plt.subplots()
ax.plot(fpr, tpr)

### `plot_roc_curve()`

You can also use the `plot_roc_curve()` function with just your fitted model and test data.

In [ ]:
plot_roc_curve(hd_model, X_test_sc, y_test)

## Sidebar: Visualizing Threshold Changes

This [ROC Applet](https://web.archive.org/web/20210210014824/http://www.navan.name/roc/) helps  visualize how a change in the threshold corresponds to moving along the ROC curve

# Scenario: Breast Cancer Prediction

Let's evaulate a model using scikit-learn's breast cancer dataset:

In [ ]:
# Load the data
preds, target = load_breast_cancer(return_X_y=True)

# Split into train and test
X_train, X_test, y_train, y_test = train_test_split(preds, target,
                                                   random_state=42)

# Scale the data
bc_scaler = StandardScaler()
bc_scaler.fit(X_train)
X_train_sc = bc_scaler.transform(X_train)
X_test_sc = bc_scaler.transform(X_test)

# Run the model
bc_model = LogisticRegression(solver='lbfgs', max_iter=10000,
                           random_state=42)
bc_model.fit(X_train_sc, y_train)

## Task

For this example, draw the ROC curve and calculate the AUC-ROC metric. Based on the results, do you think your model would be useful for identifying patients with breast cancer?

In [ ]:
# Your work here

# Oversampling

One of the most effective strategies to handle these cases is **oversampling the minority class**. That is, I give myself more data points than I really have. I could achieve this either by [bootstrapping](https://scikit-learn.org/stable/modules/generated/sklearn.utils.resample.html) or by generating some data that is fake but close to actual data. The latter is the idea behind [SMOTE](https://imbalanced-learn.org/stable/over_sampling.html).

# Loss Functions

Another more "natural" way of measuring the quality of a classifier is just to look at the loss function, which will often be the **log loss**. In multiclass problems, we use **cross-entropy loss**. 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(hd_X, hd_y, test_size=.25,
                                                   random_state=1)

log_loss(y_test, y_hat_hd)

While such loss values are difficult to interpret on their own, they are useful for comparing models. Models with lower loss values generate probability estimates that are closer to the true values, and thus are likely to perform better on many metrics.

# Level Up: Log Loss by Hand

Log loss is generally calculated as an average per data point, and is computed as follows:

$L(y, \hat{y}) = -\frac{1}{N}\sum^N_{i=1}[y_i\ln(\hat{y_i}) + (1-y_i)\ln(1-\hat{y_i})]$,

where $y$ is the vector of true values and $\hat{y}$ is the vector of probabilities that the point in question has a correct label of 1.

- Suppose, for a given data point, that the correct prediction of the label is **0**. In that case, the contribution from that point to the sum in the loss function defined above will be $-\ln(1-\hat{y_i})$. So, the closer the prediction for that point is to 0, the closer the contribution to the sum will be to $-\ln(1)=0$. But as the prediction gets closer to 1, the closer the contribution will be to $-\ln(0)=\infty$.

- Suppose, on the other hand, that the correct prediction is **1**. In that case, the contribution from that point to the sum in the loss function defined above will be $-\ln(\hat{y_i})$. So, the closer the prediction for that point is to 1, the closer the contribution to the sum will be to $-\ln(1)=0$. But as the prediction gets closer to 0, the closer the contribution will be to $-\ln(0)=\infty$.

In [ ]:
compare = list(zip(y_test, y_prob))

In [ ]:
compare[:5]

In [ ]:
calc = [-(yi * np.log(yi_hat[1]) + (1 - yi) * np.log(yi_hat[0])) for (yi, yi_hat) in compare]
calc[:5]

In [ ]:
np.mean(calc)